In [19]:
import numpy as np
import pandas as pd 
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt

In [20]:
df = pd.read_csv("./playground-series-s4e9/train.csv",index_col='id')
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [21]:
# Extract values using regular expressions
df['horse_power'] = df['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)  # Extracts the float before 'HP'
df['tank_size'] = df['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)     # Extracts the float before 'L'
df['num_cylinders'] = df['engine'].str.extract(r'(\d+)\sCylinder').astype(float)    # Extracts the float (to handle NaN) before 'Cylinder'

df.drop('engine', axis=1, inplace=True)  # Drop the original 'engine' column

# Display the resulting DataFrame
df.head()

,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,horse_power,tank_size,num_cylinders
id,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4.0
1,Lincoln,LS V8,2002,143250,Gasoline,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9,8.0
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3,8.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0,8.0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0,4.0


In [22]:
df["transmission"].value_counts()

transmission
A/T                                                   49904
8-Speed A/T                                           20645
Transmission w/Dual Shift Mode                        19255
6-Speed A/T                                           18044
6-Speed M/T                                           11998
7-Speed A/T                                           11124
Automatic                                             10691
8-Speed Automatic                                      8431
10-Speed A/T                                           8044
9-Speed A/T                                            3866
5-Speed A/T                                            3217
10-Speed Automatic                                     3164
6-Speed Automatic                                      2799
4-Speed A/T                                            2546
5-Speed M/T                                            2409
9-Speed Automatic                                      2325
CVT Transmission           

In [ ]:
# Initialize all attributes to False
df['automatic'] = False
df['manual'] = False
df['dual'] = False

# Categorize as 'dual' if it contains 'Dual'
df.loc[df['transmission'].str.contains(r'\b(Dual|At/Mt)\b', case=False, na=False), 'dual'] = True

# Categorize as 'automatic' if it contains AT, A/T, CVT, or Automatic (but not Dual)
df.loc[
    (df['transmission'].str.contains(r'\b(AT|A/T|CVT|Automatic)\b', case=False, na=False)) &
    ~df['dual'],
    'automatic'
] = True

# Categorize as 'manual' if it contains MT, M/T, or Manual (but not Dual)
df.loc[
    (df['transmission'].str.contains(r'\b(MT|M/T|Manual)\b', case=False, na=False)) &
    ~df['dual'],
    'manual'
] = True

df.head()

/tmp/ipykernel_104008/827697106.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['transmission'].str.contains(r'\b(Dual|At/Mt)\b', case=False, na=False), 'dual'] = True
/tmp/ipykernel_104008/827697106.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (df['transmission'].str.contains(r'\b(AT|A/T|CVT|Automatic)\b', case=False, na=False)) &
/tmp/ipykernel_104008/827697106.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (df['transmission'].str.contains(r'\b(MT|M/T|Manual)\b', case=False, na=False)) &


,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,horse_power,tank_size,num_cylinders,automatic,manual,dual
id,,,,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4.0,True,False,False
1,Lincoln,LS V8,2002,143250,Gasoline,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9,8.0,True,False,False
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3,8.0,True,False,False
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0,8.0,False,False,True
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0,4.0,True,False,False


In [33]:
# Identify rows with more than 1 True value in the attributes
rows_with_multiple_true = df[df[['automatic', 'manual', 'dual']].sum(axis=1) > 1]

# Print the problematic rows
print(rows_with_multiple_true[['transmission', 'automatic', 'manual', 'dual']])

Empty DataFrame
Columns: [transmission, automatic, manual, dual]
Index: []


In [8]:
df.isna().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [11]:
df["fuel_type"].value_counts()

fuel_type
Gasoline          165940
Hybrid              6832
E85 Flex Fuel       5406
Diesel              3955
–                    781
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64

## Handling Null values in the fuel_type and accident attributes

Since the number of NUlls is very low (3%, 1.5% of the data respectively) we have 2 options:

Option 1 - drop those entries and train the model without them.

Option 2 - Replacing them with another value:
1. The most frequent value in the dataset.
2. A value indicating "unknown".

In [ ]:
# Option 1 - Fuel Type

df1 = df.copy()
df1 = df1[(df1['fuel_type'] != 'not supported')]
df1 = df1[(df1['fuel_type'] != '–')]
df1 = df1.dropna(subset=['fuel_type'])

In [55]:
# Option 1 - Accident

df1 = df1.dropna(subset=['accident'])

In [56]:
df1.isna().sum()    

brand               0
model               0
model_year          0
milage              0
fuel_type           0
engine              0
transmission        0
ext_col             0
int_col             0
accident            0
clean_title     18448
price               0
dtype: int64

In [ ]:
# Option 2 - Fuel Type

df2 = df.copy()
df2["fuel_type"] = df2["fuel_type"].replace('not supported', "Gasoline")
df2["fuel_type"] = df2["fuel_type"].replace('–', "Gasoline")
df2["fuel_type"] = df2["fuel_type"].replace(pd.NA, "Gasoline")

In [57]:
# Option 2 - Accident

df2 = df2.fillna("Unknown")

## Handling Null values in the clean_title attribute

Since the number of NUlls is more significant, we have 2 different options:

Option 1 - drop those entries, since the column would only have one value remaining, it would not contribute to the model training - we would have to drop the attribute entirely.

Option 2 - Replacing them with another value - "No".

We will choose option 2.

In [60]:
df1 = df1.fillna("No")
df2 = df2.fillna("No")

In [61]:
df1.isna().sum()   

brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression

# Define the columns
ordinal_cols = ['clean_title']
onehot_cols = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident']
numeric_cols = ['model_year', 'milage', 'price']

# Define the transformers
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

onehot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

numeric_transformer = SimpleImputer(strategy='mean')

# Combine the transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', ordinal_transformer, ordinal_cols),
        ('onehot', onehot_transformer, onehot_cols),
        ('num', numeric_transformer, numeric_cols)
    ])

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the pipeline to the data
X = df.drop(columns=['price'])
y = df['price']
pipeline.fit(X, y)

In [70]:
df["int_col"].value_counts()

int_col
Black           107665
Beige            24492
Gray             21201
Brown             5810
Red               5145
                 ...  
Bianco Polar         7
Classic Red          7
Cobalt Blue          6
WHITE                6
ORANGE               4
Name: count, Length: 156, dtype: int64